## Assignment 9.1

In [1]:
import os
import shutil
import json
from pathlib import Path

import pandas as pd
import warnings
warnings.filterwarnings('ignore')

from kafka import KafkaProducer, KafkaAdminClient
from kafka.admin.new_topic import NewTopic
from kafka.errors import TopicAlreadyExistsError

from pyspark import SparkConf
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark import SparkConf
from pyspark.sql.functions import window, from_json, col
from pyspark.sql.types import StringType, TimestampType, DoubleType, StructField, StructType
from pyspark.sql.functions import udf

current_dir = Path(os.getcwd()).absolute()
checkpoint_dir = current_dir.joinpath('checkpoints')
locations_checkpoint_dir = checkpoint_dir.joinpath('locations')
accelerations_checkpoint_dir = checkpoint_dir.joinpath('accelerations')

if locations_checkpoint_dir.exists():
    shutil.rmtree(locations_checkpoint_dir)
    
if accelerations_checkpoint_dir.exists():
    shutil.rmtree(accelerations_checkpoint_dir)

locations_checkpoint_dir.mkdir(parents=True, exist_ok=True)
accelerations_checkpoint_dir.mkdir(parents=True, exist_ok=True)

### Configuration Parameters 

> **TODO:** Change the configuration prameters to the appropriate values for your setup.

In [3]:
config = dict(
    bootstrap_servers=['kafka.kafka.svc.cluster.local:9092'],
    first_name='Jake',
    last_name='Meyer'
)

config['client_id'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)
config['topic_prefix'] = '{}{}'.format(
    config['last_name'], 
    config['first_name']
)

config['locations_topic'] = '{}-locations'.format(config['topic_prefix'])
config['accelerations_topic'] = '{}-accelerations'.format(config['topic_prefix'])
config['simple_topic'] = '{}-simple'.format(config['topic_prefix'])

config

{'bootstrap_servers': ['kafka.kafka.svc.cluster.local:9092'],
 'first_name': 'Jake',
 'last_name': 'Meyer',
 'client_id': 'MeyerJake',
 'topic_prefix': 'MeyerJake',
 'locations_topic': 'MeyerJake-locations',
 'accelerations_topic': 'MeyerJake-accelerations',
 'simple_topic': 'MeyerJake-simple'}

### Create Topic Utility Function

The `create_kafka_topic` helps create a Kafka topic based on your configuration settings.  For instance, if your first name is *John* and your last name is *Doe*, `create_kafka_topic('locations')` will create a topic with the name `DoeJohn-locations`.  The function will not create the topic if it already exists. 

In [4]:
def create_kafka_topic(topic_name, config=config, num_partitions=1, replication_factor=1):
    bootstrap_servers = config['bootstrap_servers']
    client_id = config['client_id']
    topic_prefix = config['topic_prefix']
    name = '{}-{}'.format(topic_prefix, topic_name)
    
    admin_client = KafkaAdminClient(
        bootstrap_servers=bootstrap_servers, 
        client_id=client_id
    )
    
    topic = NewTopic(
        name=name,
        num_partitions=num_partitions,
        replication_factor=replication_factor
    )

    topic_list = [topic]
    try:
        admin_client.create_topics(new_topics=topic_list)
        print('Created topic "{}"'.format(name))
    except TopicAlreadyExistsError as e:
        print('Topic "{}" already exists'.format(name))
    
create_kafka_topic('simple')

Topic "MeyerJake-simple" already exists


In [6]:
spark = SparkSession\
    .builder\
    .appName("Assignment 9")\
    .getOrCreate()


df_locations = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka.kafka.svc.cluster.local:9092") \
  .option("subscribe", config['locations_topic']) \
  .option("startingOffsets", "earliest") \
  .load()

In [7]:
spark.version

'3.4.0'

In [8]:
## Understand what df_Locations looks like.
print(df_locations)

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]


In [9]:
## Understand what the schema looks like.
print(df_locations.printSchema())

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

None


**TODO:** Create a data frame called `df_accelerations` that reads from the accelerations topic you published to in assignment 8.  In order to read data from this topic, make sure that you are running the notebook you created in assignment 8 that publishes acceleration and location data to the `LastnameFirstname-simple` topic. 

In [10]:
spark = SparkSession\
    .builder\
    .appName("Assignment 9")\
    .getOrCreate()


## Try following similar suite as df_locations setup as above, but for accelerations.
df_accelerations = spark \
  .readStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka.kafka.svc.cluster.local:9092") \
  .option("subscribe", config['accelerations_topic']) \
  .option("startingOffsets", "earliest") \
  .load()

In [11]:
## Understand what df_accelerations looks like.
print(df_accelerations)

DataFrame[key: binary, value: binary, topic: string, partition: int, offset: bigint, timestamp: timestamp, timestampType: int]


In [12]:
## Understand what the schema looks like.
print(df_accelerations.printSchema())

root
 |-- key: binary (nullable = true)
 |-- value: binary (nullable = true)
 |-- topic: string (nullable = true)
 |-- partition: integer (nullable = true)
 |-- offset: long (nullable = true)
 |-- timestamp: timestamp (nullable = true)
 |-- timestampType: integer (nullable = true)

None


**TODO:** Create two streaming queries, `ds_locations` and `ds_accelerations` that publish to the `LastnameFirstname-simple` topic. See http://spark.apache.org/docs/latest/structured-streaming-programming-guide.html#starting-streaming-queries and http://spark.apache.org/docs/latest/structured-streaming-kafka-integration.html for more information. 

In [13]:
## As specified in the resources, try following code examples for setting up the streaming queries
## for ds_locations and ds_accelerations.

ds_locations = df_locations \
  .selectExpr("CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka.kafka.svc.cluster.local:9092") \
  .option("subscribe", config['locations_topic']) \
  .option("checkpointLocation", locations_checkpoint_dir) \
  .start()

ds_accelerations = df_accelerations \
  .selectExpr("CAST(value AS STRING)") \
  .writeStream \
  .format("kafka") \
  .option("kafka.bootstrap.servers", "kafka.kafka.svc.cluster.local:9092") \
  .option("topic", config['simple_topic']) \
  .option("checkpointLocation", accelerations_checkpoint_dir) \
  .start()

try:
    ds_locations.awaitTermination()
    ds_accelerations.awaitTermination()
except KeyboardInterrupt:
    print("STOPPING STREAMING DATA")

23/05/13 14:14:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/05/13 14:14:24 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.
23/05/13 14:14:25 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
23/05/13 14:14:25 WARN AdminClientConfig: The configuration 'key.deserializer' was supplied but isn't a known config.
23/05/13 14:14:25 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
23/05/13 14:14:25 WARN AdminClientConfig: The configuration 'enable.auto.commit' was supplied but isn't a known config.
23/05/13 14:14:25 WARN AdminClientConfig: The configuration 'value.deserializer' was supplied but isn't a known config.
23/05/13 14:14:25 WARN AdminClientConfig: The configuration 'max.poll.records' was supplied but isn't a known config

StreamingQueryException: [STREAM_FAILED] Query [id = 1f6412a2-7f18-4af9-a218-3499e16d1422, runId = 77b8b955-4e7c-4a03-afe6-5342deb06cf3] terminated with exception: org/apache/kafka/clients/admin/OffsetSpec

In [12]:
print(ds_locations)

In [13]:
print(type(ds_locations))

<class 'pyspark.sql.streaming.query.StreamingQuery'>


In [14]:
print(ds_accelerations)

In [15]:
print(type(ds_accelerations))

<class 'pyspark.sql.streaming.query.StreamingQuery'>
